In [32]:
import pandas as pd
import numpy as np
import os
from pathlib import Path


df_inventory = pd.read_csv('Inventory_mikelele_2025.May.30.csv')
# display(df_inventory.loc[(df_inventory['Name'] == 'Lightning Bolt')])

df_inventory = df_inventory.drop(
    columns=[
        'Tradelist Count',
        'Decks Count Built',
        'Decks Count All',
        'Type',
        'Cost',
        'Rarity',
        'Price',
        'Image URL',
        'Edition',
        'Edition Code',
        'Card Number',
        'Condition',
        'Language',
        'Foil',
        'Signed',
        'Artist Proof',
        'Altered Art',
        'Misprint',
        'Promo',
        'Textless',
        'Printing Id', 
        'Printing Note',
        'Tags',
        'My Price',
        'Last Updated',
        'TcgPlayer ID'
        ])
display(df_inventory.head())

display(df_inventory.loc[df_inventory['Name'] == 'Lightning Bolt'])

df_inventory_group = df_inventory.groupby('Name', as_index=False).agg({'Count': 'sum', 'Scryfall ID': 'first'}).rename(columns={'Count': 'sum_count'})[['Name', 'sum_count', 'Scryfall ID']]
display(df_inventory_group.loc[df_inventory_group['Name'] == 'Lightning Bolt'])

,Count,Name,Scryfall ID
0,4,Abandoned Campground,ee0565f5-ebdb-43f9-bbb4-0485b1968937
1,3,Academy Drake,f8bacb12-da46-4b00-804f-9ff6bff452bc
2,4,Academy Journeymage,a46a65e0-66a3-4896-8acc-0ad5e9927c40
3,2,Access Tunnel,edf8eb51-9643-4c54-b38e-e7abea92bbe1
4,4,Accumulated Knowledge,ad88e5ee-0eee-47af-a7b4-9bac044e1c8c


,Count,Name,Scryfall ID
1355,4,Lightning Bolt,ccee0b4c-0cb0-4c0f-8ddc-bc74b8b97273
1356,4,Lightning Bolt,cb9b9a9d-ae4c-4e04-bf9d-cae48f01292c
1357,1,Lightning Bolt,9521375e-0bc1-45ef-b513-6d332a25f9d2


,Name,sum_count,Scryfall ID
1295,Lightning Bolt,9,ccee0b4c-0cb0-4c0f-8ddc-bc74b8b97273


In [18]:
directory = Path('decks')

df_decks = pd.DataFrame()

for file in directory.iterdir():

    if file.is_file():
        deck_name = file.name.split('_')[0]
        temp_df = pd.read_csv(file)
        deck_name_arr = list([deck_name] * len(temp_df))
        temp_df.insert(0, 'deck', deck_name_arr)

        temp_df = temp_df.drop(columns=['Rarity', 'Section', 'Last Updated', 'TcgPlayer ID'])
        #display(temp_df.head())
        df_decks = pd.concat([df_decks, temp_df])


# display(df_decks)

# print(df_decks['deck'].unique())
display(df_decks.loc[df_decks['Name'] == 'Lightning Bolt'])

,deck,Count,Name,Type,Cost,Price,Image URL,Scryfall ID
11,Goblins,4,Lightning Bolt,Instant,{R},$0.78,https://s.deckbox.org/system/images/mtg/cards/...,32a180dd-9641-4546-8940-9b21084f0c71
7,Kiln Fiend,2,Lightning Bolt,Instant,{R},$0.78,https://s.deckbox.org/system/images/mtg/cards/...,32a180dd-9641-4546-8940-9b21084f0c71
7,Rakdos Madness,4,Lightning Bolt,Instant,{R},$0.78,https://s.deckbox.org/system/images/mtg/cards/...,32a180dd-9641-4546-8940-9b21084f0c71
